In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!pip install python-terrier==0.10.0 nltk scikit-learn lightgbm xgboost fastrank

In [ ]:
# !pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR

In [ ]:
import pandas as pd
import json
import pyterrier as pt
import random

In [ ]:
#UNCOMMENT THE FOLLOWING LINE TO USE EITHER THE TVR DATASET OR THE QVH DATASET

dataset_choice = "TVR"
# dataset_choice = "QVH"

In [ ]:
# Paths to JSONL files on Colab
# if dataset_choice == "TVR":
#    jsonl_train_path = '/content/drive/MyDrive/IR/text_data/tvr_train_release.jsonl'
#    jsonl_val_path = '/content/drive/MyDrive/IR/text_data/tvr_val_release.jsonl'
#    subs_path = '/content/drive/MyDrive/IR/text_data/tvqa_preprocessed_subtitles.jsonl'

# elif dataset_choice == "QVH":
#    jsonl_train_path = "/content/drive/MyDrive/IR/text_data_QVH/highlight_train_release.jsonl"
#    subs_path = "/content/drive/MyDrive/IR/text_data_QVH/subs_train.jsonl"

In [ ]:
# Paths to JSONL files
if dataset_choice == "TVR":
    jsonl_train_path = '/kaggle/input/d/boxzhu/tvretrieval/tvr_train_release.jsonl'
    jsonl_val_path = '/kaggle/input/d/boxzhu/tvretrieval/tvr_val_release.jsonl'
    subs_path = '/kaggle/input/d/boxzhu/tvretrieval/tvqa_preprocessed_subtitles.jsonl'

elif dataset_choice == "QVH":
    jsonl_train_path = "text_data_QVH/highlight_train_release.jsonl"
    subs_path = "text_data_QVH/subs_train.jsonl"

In [ ]:
# #Paths to JSONL files
# if dataset_choice == "TVR":
#     jsonl_train_path = 'text_data/tvr_train_release.jsonl'
#     jsonl_val_path = 'text_data/tvr_val_release.jsonl'
#     subs_path = 'text_data/tvqa_preprocessed_subtitles.jsonl'

# elif dataset_choice == "QVH":
#     jsonl_train_path = "text_data_QVH/highlight_train_release.jsonl"
#     subs_path = "text_data_QVH/subs_train.jsonl"

In [ ]:
# Load subtitles into a dictionary for quick access
# Load subtitles into a dictionary for quick access
subtitles_dict = {}
if dataset_choice == "TVR":
    with open(subs_path, 'r') as subs_file:
        for line in subs_file:
            sub_data = json.loads(line)
            subtitles_dict[sub_data['vid_name']] = sub_data['sub']
elif dataset_choice == "QVH":
    with open(subs_path, 'r') as subs_file:
        for line in subs_file:
            sub_data = json.loads(line)
            triple = sub_data['vid'].split("_")
            name = sub_data['vid']
            #turn the list name into a string
            name = "".join(name)
            if name not in subtitles_dict:
                subtitles_dict[name] = [(float(triple[-2]) + sub_data["relevant_windows"][0][0], float(triple[-2]) + sub_data["relevant_windows"][0][1], sub_data['query'])]
            else:
                subtitles_dict[name].append((float(triple[-2]) + sub_data["relevant_windows"][0][0], float(triple[-2]) + sub_data["relevant_windows"][0][1], sub_data['query']))

            # print(subtitles_dict[name])


In [ ]:
# Function to find matching subtitles in TVR case
def find_matching_subtitles(vid_name, ts_range, subtitles_dict):
    matching_subs = []
    if vid_name in subtitles_dict:
        for subtitle in subtitles_dict[vid_name]:
            if (ts_range[0] <= subtitle['start'] <= ts_range[1]) or (ts_range[0] <= subtitle['end'] <= ts_range[1]) or (subtitle['start'] <= ts_range[0] and subtitle['end'] >= ts_range[1]):
                matching_subs.append(subtitle['text'])
    return matching_subs

In [ ]:
def parse_jsonl_TVR(jsonl_path, split_type):
    # Initialize empty lists for your data
    queries_data = []
    documents_data = []
    query_rankings_data = []

    with open(jsonl_path, 'r') as file:
        for idx, line in enumerate(file):
            data = json.loads(line)
            # drop non text-based queries
            if data['type'] not in ['t']:
                continue

            # Find matching subtitles
            matching_subs = find_matching_subtitles(data['vid_name'], data['ts'], subtitles_dict)

            if matching_subs == []:
                continue

            # Extract data for the Query Set DataFrame
            queries_data.append({'qid': str(data['desc_id']), 'query': data['desc']})

            # Extract data for the Documents Set DataFrame, including matching subtitles
            documents_data.append({'docno': split_type + str(idx), 'vid_name': data['vid_name'], 'ts': data['ts'],
                                'duration': data['duration'], 'type': data['type'], 'text': "".join(matching_subs)})

            # Extract data for the Query Rankings DataFrame
            query_rankings_data.append({'qid': str(data["desc_id"]), 'query': data['desc'], 'docno': split_type + str(idx), 'rank': 1, 'score': 1.0})

    return queries_data, documents_data, query_rankings_data

def parse_jsonl_QVH(jsonl_path):
    queries_data = []
    documents_data = []
    query_rankings_data = []
    with open(jsonl_path, 'r') as file:
        for idx,line in enumerate(file):

            # Load the JSON object from the line
            data = json.loads(line)

            triple = data["vid"].split("_")
            document_name = triple[0:-2]
            document_name = data["vid"]
            start_time = float(triple[-2])
            end_time = float(triple[-1])

            if document_name not in subtitles_dict:
                #print("Document not found in subtitles: ", document_name)
                continue

            subs = []
            for relevant_window in data["relevant_windows"]:
                ts = [start_time+relevant_window[0], start_time+relevant_window[1]]
                subs.extend([sub for sub in subtitles_dict[document_name] if sub[0] <= ts[1] and ts[0] <= sub[1]])
                if len(subs) == 0:
                    #print("No subtitles found for ", document_name, " at time ", ts)
                    continue

            documents_data.append({"docno": str(idx), "vid_name": document_name, "ts": [start_time, end_time], "duration" : data["duration"], "text": " ".join([sub[2] for sub in subs])})
            queries_data.append({"qid" : str(data["qid"]), "query": data["query"]})
            query_rankings_data.append({"qid": str(data["qid"]), "query": data["query"], "docno": str(idx), "rank": 1, "score": 1.0})

    return queries_data, documents_data, query_rankings_data

In [ ]:

if dataset_choice == "TVR":
    queries_data_train, documents_data_train, query_rankings_data_train = parse_jsonl_TVR(jsonl_train_path, "t")
    queries_data_val, documents_data_val, query_rankings_data_val = parse_jsonl_TVR(jsonl_val_path, "v")
    #have to create a test set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_test = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_test]
    queries_data_test = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_test]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_test]
    documents_data_test = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_test]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_test]

elif dataset_choice == "QVH":
    queries_data_train, documents_data_train, query_rankings_data_train = parse_jsonl_QVH(jsonl_train_path)
    #have to create a val set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_val = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_val]
    queries_data_val = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_val]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_val]
    documents_data_val = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_val]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_val]
    #have to create a test set; to do it, extract a random 10% of the train set
    random.seed(42)
    query_rankings_data_test = random.sample(query_rankings_data_train, int(len(query_rankings_data_train)*0.1))
    query_rankings_data_train = [query for query in query_rankings_data_train if query not in query_rankings_data_test]
    queries_data_test = [query for query in queries_data_train if query["qid"] in [query["qid"] for query in query_rankings_data_test]]
    queries_data_train = [query for query in queries_data_train if query not in queries_data_test]
    documents_data_test = [doc for doc in documents_data_train if doc["docno"] in [query["docno"] for query in query_rankings_data_test]]
    documents_data_train = [doc for doc in documents_data_train if doc not in documents_data_test]



In [ ]:
# Create DataFrames for the Query Set, Documents Set, and Query Rankings
queries_train_df = pd.DataFrame(queries_data_train)
documents_train_df = pd.DataFrame(documents_data_train)

queries_val_df = pd.DataFrame(queries_data_val)
documents_val_df = pd.DataFrame(documents_data_val)

queries_test_df = pd.DataFrame(queries_data_test)
documents_test_df = pd.DataFrame(documents_data_test)

q_rels = pd.concat([pd.DataFrame(query_rankings_data_train), pd.DataFrame(query_rankings_data_val), pd.DataFrame(query_rankings_data_test)]).reset_index(drop=True)

#print length of the dataframes
print("Train set:")
print("Queries: ", len(queries_train_df))
print("Documents: ", len(documents_train_df))

print("Val set:")
print("Queries: ", len(queries_val_df))
print("Documents: ", len(documents_val_df))

print("Test set:")
print("Queries: ", len(queries_test_df))
print("Documents: ", len(documents_test_df))

print("Query Rankings: ", len(q_rels))



### First Stage Retrieval [TODO: BOX]
The following part of the code will define three different first stage retrieval pipelines as an input for the trained model.

In [ ]:
!ls -la /usr/lib/jvm/java-11-openjdk-amd64
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

if not pt.started():
    pt.init()

In [ ]:
# Create an index
from pathlib import Path

indexer = pt.IterDictIndexer(
    "./index_path/",
    meta={
        "docno": 64,
        "vid_name": 64,
        "text": 131072,
    },
    stemmer="porter",
    stopwords="terrier",
    overwrite=True,
    type=pt.index.IndexingType.MEMORY,
)

In [ ]:
joint_documents_set_df = pd.concat([documents_train_df, documents_val_df, documents_test_df])

print("Length: ", len(joint_documents_set_df))

In [ ]:
indexed = indexer.index(
    joint_documents_set_df.to_dict(orient="records")
)

In [ ]:
#These are some examples, not necessarily the best ones. Experiment with different models

# Initialize BatchRetrieve with the created index and specify BM25 as the weighting model
first_stage_bm25 = pt.BatchRetrieve(
    indexed,
    wmodel="BM25",
    num_results=100,
    metadata=["docno", "vid_name", "text"]
)

# Initialize BatchRetrieve with the created index and specify LemurTF_IDF as the weighting model
first_stage_lemurtfidf = pt.BatchRetrieve(
    indexed,
    wmodel="LemurTF_IDF",
    num_results=100,
    metadata=["docno", "vid_name", "text"]
)

# Initialize BatchRetrieve with the created index and specify Hiemstra_LM as the weighting model
first_stage_hiemstra_lm = pt.BatchRetrieve(
    indexed,
    wmodel="Hiemstra_LM",
    num_results=100,
    metadata=["docno", "vid_name", "text"]
)

first_stage_dfic = pt.BatchRetrieve(
    indexed,
    wmodel="DFIC",
    num_results=100,
    metadata=['docno', 'vid_name', 'text']
)

first_stage_lgd = pt.BatchRetrieve(
    indexed,
    wmodel="LGD",
    num_results=100,
    metadata=["docno", "vid_name", "text"]
)

first_stage_in_exp_b2 = pt.BatchRetrieve(
    indexed,
    wmodel="In_expB2",
    num_results=100,
    metadata=["docno", "vid_name", "text"]
)

In [ ]:
# RDM
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(subtitles['prepared_trainqueries'])

def enhance_query_with_rdm(query):
    query_vec = vectorizer.transform([query])
    cos_similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    # Select top-N most similar subtitles to enhance the query representation
    top_n_idx = np.argsort(cos_similarities)[-5:]  # For example, top 5
    top_n_texts = subtitles.iloc[top_n_idx]['prepared_trainqueries'].values
    enhanced_query = ' '.join([query] + list(top_n_texts))
    return enhanced_query

queries['prepared_trainqueries'] = queries['prepared_trainqueries'].apply(enhance_query_with_rdm)


# Computing feature
The weighting model can be use in [pt.weighting_model.package: http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html](http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html)

In [ ]:
#We create features for the second stage using the first stage retrievers

#TF-IDF based features
lemur_tf_idf_retriever = pt.BatchRetrieve(indexed, wmodel="LemurTF_IDF")
bm25_retriever = pt.BatchRetrieve(indexed, wmodel="BM25")
tf_idf_retriever = pt.BatchRetrieve(indexed, wmodel="TF_IDF")

#Language model based features
hiem_retriever = pt.BatchRetrieve(indexed, wmodel="Hiemstra_LM")
dirichlet_retriever = pt.BatchRetrieve(indexed, wmodel="DirichletLM")

#Divergence from randomness based features
pl2_retriever = pt.BatchRetrieve(indexed, wmodel="PL2")
dph_retriever = pt.BatchRetrieve(indexed, wmodel="DPH")
dlh_retriever = pt.BatchRetrieve(indexed, wmodel="DLH")

#Can add more!
coordinate_match_retriever = pt.BatchRetrieve(indexed, wmodel="CoordinateMatch")
js_kls_retrieveer = pt.BatchRetrieve(indexed, wmodel="Js_KLs")
dfic_retriever = pt.BatchRetrieve(indexed, wmodel="DFIC")
in_exp_b2_retriever = pt.BatchRetrieve(indexed, wmodel="In_expB2")
lgd_retriever = pt.BatchRetrieve(indexed, wmodel="LGD")
mdl2_retriever = pt.BatchRetrieve(indexed, wmodel="MDL2")
ifp2_retrieveer = pt.BatchRetrieve(indexed, wmodel="IFB2")
pl2f_retriever = pt.BatchRetrieve(indexed, wmodel="PL2F")
SMF_retriever = pt.BatchRetrieve(indexed, wmodel="SingleFieldModel")
static_retriever = pt.BatchRetrieve(indexed, wmodel="StaticFeature")
xsqrA_retriever = pt.BatchRetrieve(indexed, wmodel="XSqrA_M")
di_retriever = pt.BatchRetrieve(indexed, wmodel="Dl")
dlh_retrieveer = pt.BatchRetrieve(indexed, wmodel="DLH")
ifb2_retriever = pt.BatchRetrieve(indexed, wmodel="IFB2")
in_exp_c2_retriever = pt.BatchRetrieve(indexed, wmodel="In_expC2")
in_b2_retriever = pt.BatchRetrieve(indexed, wmodel="InB2")
in_l2_retrieveer = pt.BatchRetrieve(indexed, wmodel="InL2")


In [ ]:
# PIPELINES WITH FSR AND FEATURES

#Can test many possibilities, i just put some examples here

in_exp_b2_pipeline = ~first_stage_in_exp_b2 >> (
    bm25_retriever ** hiem_retriever ** pl2_retriever ** coordinate_match_retriever ** in_exp_b2_retriever
)

in_exp_b2_pipeline_woBM25 = ~first_stage_in_exp_b2 >> (
    hiem_retriever ** pl2_retriever ** coordinate_match_retriever ** in_exp_b2_retriever
)

in_exp_b2_pipeline_wohiem = ~first_stage_in_exp_b2 >> (
    bm25_retriever ** pl2_retriever ** coordinate_match_retriever ** in_exp_b2_retriever
)

in_exp_b2_pipeline_wocoodinate = ~first_stage_in_exp_b2 >> (
    bm25_retriever ** hiem_retriever ** pl2_retriever ** in_exp_b2_retriever
)
#NOTE: i believe we should not use the same first stage retriever for features AND for the first stage retrieval

In [ ]:
# Prepare the queries for the pipeline, remove special characters and extra spaces
prepared_trainqueries = queries_train_df
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_trainqueries['query'] = prepared_trainqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_train_qrels = pd.DataFrame(query_rankings_data_train)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_train_qrels['query'] = prepared_train_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_train_qrels['label'] = prepared_train_qrels['score']

prepared_train_qrels['label'] = prepared_train_qrels['label'].astype(int)

prepared_val_qrels = pd.DataFrame(query_rankings_data_val)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_val_qrels['query'] = prepared_val_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_val_qrels['label'] = prepared_val_qrels['score']

prepared_val_qrels['label'] = prepared_val_qrels['label'].astype(int)

prepared_test_qrels = pd.DataFrame(query_rankings_data_test)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_test_qrels['query'] = prepared_test_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_test_qrels['label'] = prepared_test_qrels['score']

prepared_test_qrels['label'] = prepared_test_qrels['label'].astype(int)

prepared_qrels = q_rels
prepared_qrels['query'] = prepared_qrels['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_qrels['query'] = prepared_qrels['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_qrels['query'] = prepared_qrels['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

prepared_qrels['label'] = prepared_qrels['score']

prepared_qrels['label'] = prepared_qrels['label'].astype(int)

prepared_valqueries = queries_val_df.reset_index()
prepared_valqueries['query'] = prepared_valqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_valqueries['query'] = prepared_valqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)

prepared_valqueries['query'] = prepared_valqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()

#test set
prepared_testqueries = queries_test_df.reset_index()
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace('[\'"?!]', ' ', regex=True)
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace(r'[^\w\s]', ' ', regex=True)
prepared_testqueries['query'] = prepared_testqueries['query'].str.replace(r'\s+', ' ', regex=True).str.strip()


In [ ]:
# pt.Experiment(
#     names = ["BM25","Hiemstra"],
#     retr_systems = [bm25_pipeline, hiem_lm_pipeline],
#     topics = prepared_valqueries,
#     qrels = prepared_val_qrels,
#     eval_metrics = ["map", "ndcg_cut_10", "P_10"],
# )

Storage format: .csv or trec

trec(https://pyterrier.readthedocs.io/en/latest/io.html): The pt io format, but it doesn't contain feature.

In [ ]:
from sklearn.svm import SVR
import xgboost as xgb
import fastrank

# index = pt.IndexFactory.of("./index_path")

fsr_pipelines = [
    {
       'pipe': in_exp_b2_pipeline,
       'name': 'In Exp  B2'
    },
    {
       'pipe': in_exp_b2_pipeline_woBM25,
       'name': 'In Exp  B2_woBM25'
    },
    {
       'pipe': in_exp_b2_pipeline_wohiem,
       'name': 'In Exp  B2_wohiem'
    },
    {
       'pipe': in_exp_b2_pipeline_wocoodinate,
       'name': 'In Exp  B2_wocoodinate'
    },
    
]

learned_models = [
#     {
#         'model': SVR(),
#         'form': 'reg',
#         'name': 'SVR',
#     },
#     {
#         'model': xgb.XGBRanker(tree_method="hist", objective="rank:ndcg"),
#         'form': 'ltr',
#         'name': 'XGBoost (NDCG)',
#     },
    #{
    #    'model': xgb.XGBRanker(tree_method="hist", lambdarank_num_pair_per_sample=8, objective="rank:pairwise", lambdarank_pair_method="topk"),
    #    'form': 'ltr',
    #    'name': 'XGBoost (Pariwise)'
    #}
    # {
    #     'model': xgb.XGBRanker(tree_method="hist", objective="rank:map"),
    #     'form': 'ltr',
    #     'name': 'XGBoost (MAP)',
    # },
#     {
#        'model': fastrank.TrainRequest.coordinate_ascent(),
#        'form': 'fastrank',
#        'name': 'FastRank Coordinate Ascent',
#     },
    {
       'model': fastrank.TrainRequest.random_forest(),
       'form': 'fastrank',
       'name': 'FastRank Random Forest',
    }
  ]

In [ ]:
# from pyterrier.measures import *

# trained_models = [first_stage_bm25, first_stage_lemurtfidf, first_stage_hiemstra_lm, first_stage_dfic,  first_stage_lgd, first_stage_in_exp_b2]
# names = ['BM25', 'LemurTF_IDF', 'Hiemstra LM', 'DFIC', 'LGD', 'In Exp  B2']
# eval_metrics = [
#     nDCG @ 1, nDCG @ 3, nDCG @ 5, nDCG @ 10, nDCG @ 30,
#     RR @ 1,   RR @ 3,   RR @ 5,   RR @ 10, RR @ 30,
#     MAP, NumRelRet, Judged @ 1,Judged @ 3, Judged @ 5
# ]

# for fsr in fsr_pipelines:
#     for model in learned_models:
#         names.append(f"{fsr['name']} >> {model['name']}")
#         print(names[-1])
#         if 'form' in model:
#           pipe = fsr['pipe'] >> pt.ltr.apply_learned_model(model['model'], form=model['form'])
#         else:
#           pipe = fsr['pipe'] >> pt.ltr.apply_learned_model(model['model'])
#         pipe.fit(
#             prepared_trainqueries,
#             prepared_train_qrels,
#             prepared_valqueries,
#             prepared_val_qrels
#         )
#         trained_models.append(pipe)

In [ ]:
from pyterrier.measures import *

trained_models = [first_stage_in_exp_b2]
names = ['In Exp  B2']
eval_metrics = [
    Recall @ 1,   Recall @ 3,    Recall @ 5,    Recall @ 10, 
    Recall @ 20,  Recall @ 30,   Recall @ 50,   Recall @ 100,
]

for fsr in fsr_pipelines:
    for model in learned_models:
        names.append(f"{fsr['name']} >> {model['name']}")
        print(names[-1])
        if 'form' in model:
          pipe = fsr['pipe'] >> pt.ltr.apply_learned_model(model['model'], form=model['form'])
        else:
          pipe = fsr['pipe'] >> pt.ltr.apply_learned_model(model['model'])
        pipe.fit(
            prepared_trainqueries,
            prepared_train_qrels,
            prepared_valqueries,
            prepared_val_qrels
        )
        trained_models.append(pipe)
        
        

In [ ]:
from pyterrier.measures import nDCG, RR, MAP

pt.Experiment(
    trained_models,
    prepared_testqueries,
    prepared_test_qrels,
    names=names,
    eval_metrics=eval_metrics,
)


In [ ]:
from pyterrier.measures import nDCG, RR, MAP

pt.Experiment(
    trained_models,
    prepared_valqueries,
    prepared_val_qrels,
    names=names,
    eval_metrics=eval_metrics,
)

In [ ]:
from pyterrier.measures import nDCG, RR, MAP

pt.Experiment(
    trained_models,
    prepared_trainqueries[:1000],
    prepared_train_qrels[:1000],
    names=names,
    eval_metrics=eval_metrics,
)